# Test Intent Classifier

In [ ]:
from training.training_data_generator import TrainingDataGenerator
from source.intent_classifier import IntentClassifier

# Initialize training data generator
generator = TrainingDataGenerator(
    question_bank_path='training/question_bank.md', 
    trees_csv_path='data/trees_df.csv', 
    patches_csv_path='data/patches_df.csv'
)

# Generate test questions
test_questions = generator.generate_dataset()['text']

# Initialize intent classifier
classifier = IntentClassifier()

# Predict intent for each question
for question in test_questions:
    intent_classifier_result = classifier.predict(question)
    print(f"Question: {question}")
    print(f"Intent: {intent_classifier_result['intent']} ({intent_classifier_result['confidence']:.2%} confidence)\n")

Question: What should I give the farmer for yew?
Intent: payment (80.70% confidence)

Question: What should I grow at level 39?
Intent: tree_recommendations (82.79% confidence)

Question: What level is needed to grow orange trees?
Intent: level_requirements (73.40% confidence)

Question: What do I pay the farmer to protect banana?
Intent: payment (82.54% confidence)

Question: How much time does maple need to grow?
Intent: growth_time (74.48% confidence)

Question: When will magic trees be ready?
Intent: growth_time (71.35% confidence)

Question: What should I give the farmer for palm?
Intent: payment (81.84% confidence)

Question: What tree is best at level 42?
Intent: tree_recommendations (82.81% confidence)

Question: How much time does oak need to grow?
Intent: growth_time (74.14% confidence)

Question: How do I get to the brimhaven patch?
Intent: transportation (77.19% confidence)

Question: What is the fastest way to the nemus retreat patch?
Intent: transportation (73.32% confide

# Test Answer Retriever

In [1]:
from training.training_data_generator import TrainingDataGenerator
from source.answer_retriever import AnswerRetriever

# Initialize training data generator
generator = TrainingDataGenerator(
    question_bank_path='training/question_bank.md', 
    trees_csv_path='data/trees_df.csv', 
    patches_csv_path='data/patches_df.csv'
)

# Generate test questions
test_questions = generator.generate_dataset()['text']

# Initialize answer retriever
answer_retriever = AnswerRetriever()

# Get the answer for each question
for question in test_questions:
    answer, intent, lookup_value = answer_retriever.get_answer(question)
    print(f"Question: {question}")
    print(f"Intent: {intent}")
    print(f"Lookup Value: {lookup_value}")
    print(f"Answer: {answer}")
    print('')

Question: What payment is needed for palm trees?
Intent: payment
Lookup Value: ['palm']
Answer: papaya fruit

Question: Which are the best trees at level 75?
Intent: tree_recommendations
Lookup Value: ['75']
Answer: magic

Question: At what level can I plant pineapple?
Intent: level_requirements
Lookup Value: ['pineapple']
Answer: 51

Question: What do I pay the farmer to protect orange?
Intent: payment
Lookup Value: ['orange']
Answer: basket of strawberries

Question: How long until palm is fully grown?
Intent: growth_time
Lookup Value: ['palm']
Answer: 960

Question: How long do papaya trees take to grow?
Intent: growth_time
Lookup Value: ['papaya']
Answer: 960

Question: How do I protect willow trees?
Intent: payment
Lookup Value: ['willow']
Answer: basket of apples

Question: Which tree should I grow at level 42?
Intent: tree_recommendations
Lookup Value: ['42']
Answer: curry

Question: How long does dragonfruit take to grow?
Intent: growth_time
Lookup Value: ['dragonfruit']
Answer